# Julia Academy

# 4. Grouping Rows in a Dataframe

**Bogumił Kamiński**

**Source:** https://github.com/JuliaAcademy/DataFrames/blob/main/4.%20Grouping%20data%20frames.ipynb

We will learn to how to group rows in a dataframe.

The data we will use is the `auto2.csv` and `auto2.arrow` files created from the previous notebook, [3 Working with Textfile](3_working_with_textfiles.ipynb):


In [1]:
using DataFrames
using CSV
using Arrow
using Statistics
using FreqTables
using Pipe

In [2]:
ENV["LINES"], ENV["COLUMNS"] = 15, 200

(15, 200)

Let's load the cleaned data as a dataframe now:

In [3]:
df = CSV.File("auto2.csv") |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii,plymouth
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina,pontiac


Why do we also have the apache arrow file? For Julia -> Julia IO, Apache Arrow is the preferred format for storing dataframes as files:

In [5]:
df2 = Arrow.Table("auto2.arrow") |> DataFrame
df == df2

true

But note that to ensure maximum speed, this implementation uses its own `AbstractVector` type:

In [6]:
df2.mpg

392-element Arrow.Primitive{Float64,Array{Float64,1}}:
 18.0
 15.0
 18.0
 16.0
 17.0
  ⋮
 27.0
 44.0
 32.0
 28.0
 31.0

If we wanted these columns to be standard `Vectors`, we can just do a copy:

In [7]:
df3 = copy(df2)
df3.mpg

392-element Array{Float64,1}:
 18.0
 15.0
 18.0
 16.0
 17.0
  ⋮
 27.0
 44.0
 32.0
 28.0
 31.0

## Grouping: Group by Brand

Let's group the data by brand and see what aggregations (summarizations or _integrations_) we can do. In `DataFrames.jl` it is easy to create a grouped object which we can refer to and interpret:

In [9]:
gdf = groupby(df, :brand)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
3,15.0,8.0,400.0,150.0,3761.0,9.5,70.0,1.0,chevrolet monte carlo,chevrolet
4,28.0,4.0,140.0,90.0,2264.0,15.5,71.0,1.0,chevrolet vega 2300,chevrolet
5,17.0,6.0,250.0,100.0,3329.0,15.5,71.0,1.0,chevrolet chevelle malibu,chevrolet
6,14.0,8.0,350.0,165.0,4209.0,12.0,71.0,1.0,chevrolet impala,chevrolet
7,22.0,4.0,140.0,72.0,2408.0,19.0,71.0,1.0,chevrolet vega (sw),chevrolet
8,20.0,4.0,140.0,90.0,2408.0,19.5,72.0,1.0,chevrolet vega,chevrolet
9,13.0,8.0,350.0,165.0,4274.0,12.0,72.0,1.0,chevrolet impala,chevrolet


Essentially we have indexed the dataframe according to the `:brand` column:

In [11]:
gdf[("ford",)]  # we have to pass a tuple to the grouped object incase we group by more than one variable

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
2,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
3,21.0,6.0,200.0,85.0,2587.0,16.0,70.0,1.0,ford maverick,ford
4,10.0,8.0,360.0,215.0,4615.0,14.0,70.0,1.0,ford f250,ford
5,19.0,6.0,250.0,88.0,3302.0,15.5,71.0,1.0,ford torino 500,ford
6,14.0,8.0,351.0,153.0,4154.0,13.5,71.0,1.0,ford galaxie 500,ford
7,13.0,8.0,400.0,170.0,4746.0,12.0,71.0,1.0,ford country squire (sw),ford
8,18.0,6.0,250.0,88.0,3139.0,14.5,71.0,1.0,ford mustang,ford
9,21.0,4.0,122.0,86.0,2226.0,16.5,72.0,1.0,ford pinto runabout,ford


## Aggregate using combine:

In [16]:
brand_mpg = combine(gdf, :mpg => mean)

,brand,mpg_mean
,String,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
6,pontiac,20.0125
7,dodge,22.0607
8,toyota,28.372
9,datsun,31.113


We can specify the target column name using a second mapping:

```julia
source column  => transformation => target column
```

In [17]:
combine(gdf, :mpg => mean => :mean_mpg)

,brand,mean_mpg
,String,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
6,pontiac,20.0125
7,dodge,22.0607
8,toyota,28.372
9,datsun,31.113


Let's sort the dataframe  by mean mpg:

In [18]:
ENV["LINES"] = 50   # so we can see all the brand rows

50

In [19]:
sort!(brand_mpg, :mpg_mean, rev=true)  # highest is best

,brand,mpg_mean
,String,Float64
1,vw,39.0167
2,nissan,36.0
3,triumph,35.0
4,honda,33.7615
5,datsun,31.113
6,mazda,30.86
7,subaru,30.525
8,vokswagen,29.8
9,renault,29.6667


## FreqTables.jl

Let's try and understand what origin column means. We can use `FreqTables.jl` to help us quickly count the frequencies of all `origin` values per `brand`:

In [20]:
freqtable(df, :brand, :origin)

37×3 Named Array{Int64,2}
brand ╲ origin │ 1.0  2.0  3.0
───────────────┼──────────────
amc            │  27    0    0
audi           │   0    7    0
bmw            │   0    2    0
buick          │  17    0    0
cadillac       │   2    0    0
capri          │   1    0    0
chevroelt      │   1    0    0
chevrolet      │  43    0    0
chevy          │   3    0    0
chrysler       │   6    0    0
datsun         │   0    0   23
dodge          │  28    0    0
fiat           │   0    8    0
ford           │  48    0    0
hi             │   1    0    0
honda          │   0    0   13
maxda          │   0    0    2
mazda          │   0    0   10
mercedes       │   0    1    0
mercedes-benz  │   0    2    0
mercury        │  11    0    0
nissan         │   0    0    1
oldsmobile     │  10    0    0
opel           │   0    4    0
peugeot        │   0    8    0
plymouth       │  31    0    0
pontiac        │  16    0    0
renault        │   0    3    0
saab           │   0    4    0
subaru       

It appears that:

- 1 -> 'North America'
- 2 -> 'Europe'
- 3 -> 'Asia' 

## Pipe.jl

It appears that each brand is only assigned one origin value across its models. We can use a `split-apply-combine` approach using `Pipe.jl`. Here `_` denotes the position of the argument which must be passed from the previous step in the chain (just like `dplyr` in R)

In [27]:
orig_brand = @pipe df |>
                   groupby(_, :brand) |> 
                   combine(_, :origin => ( x -> length(unique(x)) ) => :n_origin )

,brand,n_origin
,String,Int64
1,chevrolet,1
2,buick,1
3,plymouth,1
4,amc,1
5,ford,1
6,pontiac,1
7,dodge,1
8,toyota,1
9,datsun,1


Suppose we had a lot more rows. How would we quickly check that the unique value is 1? We can use `extrema` to bound the hypothesis:

In [28]:
extrema(orig_brand.n_origin)

(1, 1)

How can we get the number of models per brand easily? Using `nrow`:

In [36]:
@pipe groupby(df, :brand) |> combine(_, nrow => :n_models)

,brand,n_models
,String,Int64
1,chevrolet,43
2,buick,17
3,plymouth,31
4,amc,27
5,ford,48
6,pontiac,16
7,dodge,28
8,toyota,25
9,datsun,23


What if we want each unique value of `origin` to be a column in the dataframe? This will be similar to `freqtable`:

In [38]:
origin_vs_brand = @pipe df |>
                        groupby(_, [:origin, :brand]) |>
                        combine(_, nrow => :n_models) |>
                        unstack(_, :brand, :origin, :n_models)

,brand,1.0,3.0,2.0
,String,Int64?,Int64?,Int64?
1,chevrolet,43,missing,missing
2,buick,17,missing,missing
3,plymouth,31,missing,missing
4,amc,27,missing,missing
5,ford,48,missing,missing
6,pontiac,16,missing,missing
7,dodge,28,missing,missing
8,toyota,missing,25,missing
9,datsun,missing,23,missing


We can replace missing values using `coalesce`

In [39]:
coalesce.(origin_vs_brand, 0)

,brand,1.0,3.0,2.0
,String,Int64,Int64,Int64
1,chevrolet,43,0,0
2,buick,17,0,0
3,plymouth,31,0,0
4,amc,27,0,0
5,ford,48,0,0
6,pontiac,16,0,0
7,dodge,28,0,0
8,toyota,0,25,0
9,datsun,0,23,0


What if we wanted to get the unique brands per origin? We can store these as a Julia list, but note that we need to use the `Ref` method to prevent `DataFrames` from broadcasting to a long-format dataframe.

In [40]:
origin_brand3 = @pipe df |> 
                      groupby(_, :origin) |>
                      combine(_, :brand => ( x -> Ref(unique(x)) ) => :origin_brands)

,origin,origin_brands
,Float64,Array…
1,1.0,"[""chevrolet"", ""buick"", ""plymouth"", ""amc"", ""ford"", ""pontiac"", ""dodge"", ""chevy"", ""hi"", ""mercury"", ""oldsmobile"", ""chrysler"", ""chevroelt"", ""capri"", ""cadillac""]"
2,3.0,"[""toyota"", ""datsun"", ""mazda"", ""toyouta"", ""maxda"", ""honda"", ""subaru"", ""nissan""]"
3,2.0,"[""volkswagen"", ""peugeot"", ""audi"", ""saab"", ""bmw"", ""opel"", ""fiat"", ""volvo"", ""renault"", ""vw"", ""mercedes-benz"", ""mercedes"", ""vokswagen"", ""triumph""]"


If we didn't use `Ref` we'd get this:

In [41]:
@pipe df |> 
      groupby(_, :origin) |>
      combine(_, :brand => unique)

,origin,brand_unique
,Float64,String
1,1.0,chevrolet
2,1.0,buick
3,1.0,plymouth
4,1.0,amc
5,1.0,ford
6,1.0,pontiac
7,1.0,dodge
8,1.0,chevy
9,1.0,hi


We can check that the (origin, brand) groups are also pairwise disjoint:

In [43]:
for i in axes(origin_brand3, 1)
    for j in i+1:nrow(origin_brand3)
        println("$i vs $j: ", intersect(origin_brand3.origin_brands[[i,j]]...))
    end
end

1 vs 2: String[]
1 vs 3: String[]
2 vs 3: String[]


If we wanted to flatten the lists in `origin_brands` so that the elements are exposed as rows in a long-format dataframe, we can do the following:

In [44]:
flatten(origin_brand3, :origin_brands)

,origin,origin_brands
,Float64,String
1,1.0,chevrolet
2,1.0,buick
3,1.0,plymouth
4,1.0,amc
5,1.0,ford
6,1.0,pontiac
7,1.0,dodge
8,1.0,chevy
9,1.0,hi
